In [12]:
import wandb
from types import SimpleNamespace
from pathlib import Path
from datasets import load_dataset

In [32]:
DATASET_AT = "capecape/wizard/lm_dataset:latest"
MODEL_NAME = "WizardLM/WizardCoder-15B-V1.0"

config = SimpleNamespace(
    dataset_at=DATASET_AT,
    model_name=MODEL_NAME,
)

In [14]:
# download artifact without creating a run
api = wandb.Api()
artifact = api.artifact(config.dataset_at, type='dataset')
artifact_dir = artifact.download()
path = Path(artifact_dir)

wandb:   1 of 1 files downloaded.  


In [17]:
dataset_file = path/'dataset.jsonl'

In [25]:
dataset = load_dataset("text", data_dir=path)

Found cached dataset text (/Users/tcapelle/.cache/huggingface/datasets/text/default-2eab0995a9c25448/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


  0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3086
    })
})

## Tokenizer

In [33]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(config.model_name)

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [34]:
dataset = dataset.map(lambda examples: tokenizer(examples["text"]), batched=True)

Map:   0%|          | 0/3086 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2073 > 2048). Running this sequence through the model will result in indexing errors
